# Análisis de Inversión Inmobiliaria en CABA

**Objetivo:** Identificar oportunidades de inversión y entender las dinámicas de valor en el mercado inmobiliario de Buenos Aires.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
import sys
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

warnings.simplefilter(action='ignore', category=FutureWarning)

load_dotenv(dotenv_path='../.env')

def get_data_from_db():
    try:
        db_user = os.getenv("DB_USER")
        db_password = os.getenv("DB_PASSWORD")
        db_host = os.getenv("DB_HOST")
        db_name = os.getenv("DB_NAME")
        db_uri = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"
        engine = create_engine(db_uri)
        query = "SELECT barrio, price_usd, superficie_total_m2, description FROM propiedades WHERE price_usd IS NOT NULL AND superficie_total_m2 > 0;"
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

df = get_data_from_db()

if df is not None:
    print(f"Se cargaron {len(df)} registros.")


## 1. Análisis de "Joyas Ocultas": Precio/m² vs. Amenities

In [ ]:
if df is not None:
    df['precio_m2'] = df['price_usd'] / df['superficie_total_m2']

    amenities = ['pileta', 'gimnasio', 'sum', 'parrilla', 'seguridad']
    for amenity in amenities:
        df[amenity] = df['description'].str.contains(amenity, case=False, na=False).astype(int)

    df['amenities_score'] = df[amenities].sum(axis=1)

    df_barrios = df.groupby('barrio').agg(
        precio_m2_promedio=('precio_m2', 'mean'),
        amenities_promedio=('amenities_score', 'mean'),
        cantidad_propiedades=('barrio', 'size')
    ).reset_index()

    df_filtrado = df_barrios[df_barrios['cantidad_propiedades'] > 100]

    plt.figure(figsize=(14, 10))
    sns.scatterplot(data=df_filtrado, x='precio_m2_promedio', y='amenities_promedio', size='cantidad_propiedades', hue='barrio', sizes=(100, 2000), legend=False)

    for i, row in df_filtrado.iterrows():
        plt.text(row['precio_m2_promedio'], row['amenities_promedio'], row['barrio'], fontsize=9)

    plt.title('Precio/m² vs. Score de Amenities por Barrio', fontsize=16)
    plt.xlabel('Precio Promedio por m² (USD)', fontsize=12)
    plt.ylabel('Score Promedio de Amenities', fontsize=12)
    plt.grid(True)
    
    output_dir = "../docs/images"
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, "joyas_ocultas.png"))
    plt.show()


## 2. Análisis de Dispersión de Mercado: Volatilidad y Oportunidad

El promedio es útil, pero no cuenta toda la historia. Un barrio con un precio promedio de USD 200,000 puede tener todas sus propiedades en un rango de 190k-210k (un mercado homogéneo) o puede tener propiedades desde 120k hasta 500k (un mercado heterogéneo y con mayor dispersión).

Un **box plot** es la herramienta perfecta para visualizar esta dispersión.

In [ ]:
if df is not None:
    top_barrios = df['barrio'].value_counts().nlargest(6).index
    df_dist = df[df['barrio'].isin(top_barrios) & (df['price_usd'] < 1000000)]

    plt.figure(figsize=(14, 8))
    sns.boxplot(data=df_dist, x='barrio', y='price_usd', palette="coolwarm")
    plt.title('Distribución de Precios en Barrios Seleccionados', fontsize=16)
    plt.xlabel('Barrio', fontsize=12)
    plt.ylabel('Precio (USD)', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "distribucion_precios_barrios.png"))
    plt.show()

### Interpretación del Box Plot

*   **Cajas Altas (Gran Dispersión):** **Palermo y Recoleta** tienen las "cajas" más altas, lo que significa que el 50% central de sus propiedades abarca un rango de precios muy amplio. Esto indica un mercado muy heterogéneo, con propiedades de diferentes calidades y precios. **Insight:** En estos barrios es donde un comprador tiene más probabilidades de encontrar propiedades significativamente por debajo de la media (outliers inferiores), que pueden representar excelentes oportunidades de compra.
*   **Cajas Cortas (Mercado Homogéneo):** **Caballito y Almagro** presentan cajas mucho más compactas. Esto sugiere que los precios son más consistentes y predecibles. Es un mercado más estable, ideal para compradores que buscan un valor seguro sin tanta volatilidad.
*   **Mediana:** La línea horizontal dentro de cada caja representa el precio mediano (el valor del medio). Vemos cómo la mediana de Recoleta es la más alta, confirmando su posicionamiento como uno de los barrios más caros.